<a href="https://colab.research.google.com/github/torontoniancoder/python_tutorial/blob/master/ML_with_linear_regression_in_Python_final_project_C06_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1372]:
import pandas as pd
import numpy as np

A. Data cleaning

In [1373]:
def clean_data(df):
    pd.set_option('mode.use_inf_as_na', True) # to recognise inf as a NaN in calculations: from here: https://stackoverflow.com/questions/58225701/change-pandas-default-fillna-setting
    # todo: replace the nan values with relevant (??) values (MEAN?) -> replaced with 0-es
    # selecting the columns for conversion from categorical data into columns of binary data
    df_temp = df[['Gender','Age','Occupation', 'City_Category','Stay_In_Current_City_Years']]
    # integer encoding
    # source for instructions: https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40
    # turning the type columns into the dummy variables
    # No nan values therefore dummy_na=False
    dummy = pd.get_dummies(df_temp, dummy_na=False, columns=['Gender','Age','Occupation', 'City_Category','Stay_In_Current_City_Years'], prefix=['Gender','Age','Occupation', 'City_Category','Stay_In_Current_City_Years'], dtype=int, drop_first=False)
    # concatenating binary data into the initial dataframe
    df = pd.concat([df,dummy], axis=1)
    # dropping the unnecessary fields
    # FieldDrop0 = ['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years','Purchase']
    # df = df.drop(FieldDrop0, axis=1)
    colnames = df.columns.values.tolist() #converting column names into a list
    # colnames
    # counting nan values and removing columns with nan values more than 50% (threshold can be modified *there -> <-*)
    for each in colnames:
        # print(type(df[each]))
        if (( df[each].isna().sum() / len(df[each]) ) > 0.5): # *-> <-* here! the threshold can be modified | source URL: https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe
            print ('Due to low data quality\n(data not available for more than 50% of data points)\nI am deleting this column: '+each)
            # *-> <-* here! in the line above the threshold must be modified if it si changed in the previous line
            del df[each]
        else:
            df[each] = df[each].fillna(0)
            df[each] = df[each].astype('string')
            pass
            # z = ( df[each].isna().sum() / len(df[each]) )*100
            # print('For column '+str(each)+' unknown datapoints are '+str(z)+'% of series')
    # df.info()
    # df = df.astype('string') # source URL: https://stackoverflow.com/questions/22005911/convert-columns-to-string-in-pandas
    # df[["D", "E"]] = df[["D", "E"]].astype(int)
    #   cleaning temporary data
    df_temp = 0
    return df

In [1374]:
df = pd.read_csv('Black_Friday.csv') #loading data into a dataframe

In [1375]:
# *** REMOVE BEFORE ASSEMBLY *** BEGIN
# df = df.iloc[:7000] # slicing dataframe for test&dev performance
# df.shape
# *** REMOVE BEFORE ASSEMBLY *** END


In [1376]:
# keeping a control copy of the source dataset in variable df0
df0 = df.copy()

In [1377]:
df = clean_data(df) # running function clean_data on a dataframe

Due to low data quality
(data not available for more than 50% of data points)
I am deleting this column: Product_Category_3


In [1378]:
# df.info()
# display(df.describe())

C. Data Modeling

In [1379]:
# source code URL: https://habr.com/ru/post/206306/
# and here, URL: https://habr.com/ru/post/202090/

# importing libraries

from pandas import read_csv, DataFrame
# from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
# from sklearn.svm import SVR
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score
# from sklearn import cross_validation, svm
# from sklearn.cross_validation import train_test_split

In [1380]:
# creating dataset with summary data per UserID

# df_c1 = df.copy() #loading data into a dataframe

In [1381]:
# df_c1 = clean_data(df_c1) #
# PreFinal_df = df.groupby('User_ID').sum()

In [1382]:
# df_c1.info()

In [1383]:
# df_c1 = df_c1.astype(str)

In [1384]:
# df_c1.info()

In [1385]:
# To begin with, we need to divide our training sample
# into the indicator that we are examining, and its defining features:
# todo: check features for cross-correlation, see source code
#  here, URL: http://espressocode.top/python-pandas-dataframe-corr/

# PreFinal_df

In [1386]:
FieldDrop_PID = ['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years']
df = df.drop(FieldDrop_PID, axis=1)

In [1387]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497996 entries, 0 to 497995
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   Marital_Status                 497996 non-null  string
 1   Product_Category_1             497996 non-null  string
 2   Product_Category_2             497996 non-null  string
 3   Purchase                       497996 non-null  string
 4   Gender_F                       497996 non-null  string
 5   Gender_M                       497996 non-null  string
 6   Age_0-17                       497996 non-null  string
 7   Age_18-25                      497996 non-null  string
 8   Age_26-35                      497996 non-null  string
 9   Age_36-45                      497996 non-null  string
 10  Age_46-50                      497996 non-null  string
 11  Age_51-55                      497996 non-null  string
 12  Age_55+                        497996 non-nu

In [1388]:
df = df.astype(float)
# df.shape()
# df.info()

In [1389]:
CorrKoeff = df.corr()

In [1390]:
# CorrKoeff

In [1391]:
colnames = 0

In [1392]:
colnames = CorrKoeff.columns.values.tolist()

In [1393]:
Feature_Drop = list(())
Feature_DropSoft = list(())

In [1394]:
# checking the correlation between the features
for each in colnames:
    i = len(CorrKoeff[each])
    for i in range(i):
        if (CorrKoeff.iloc[i][each]  > 0.8) and (CorrKoeff.index[i] != each):
            # print('\n* *XX*|||*XX* * !!WARNING!! ['+str(each)+'] IS CORRELATED WITH '+str(CorrKoeff.index[i])+' BY '+str(CorrKoeff.iloc[i][each])+' * *XX*|||*XX* *')
            Feature_Drop.append(each)
            pass
        elif (CorrKoeff.iloc[i][each]  < -0.8) and (CorrKoeff.index[i] != each):
            # print('\n- *XX*XX* - Attention: ['+str(each)+'] is correlated with ['+str(CorrKoeff.index[i])+'] by '+str(CorrKoeff.iloc[i][each])+' - *XX*XX* -\n')
            Feature_DropSoft.append(each)
            pass
        else:
            pass
    # print('----- * ----- '+str(each)+' is [COMPLETE] ----- * -----')

In [1395]:
# there is a correlation > 80% and <-80% between the features in df

In [1396]:
# todo: drop the unnecessary features (columns)

In [1397]:
# writing the Purchase data (targeted for prediction) into a separate dataset
target = df0['Purchase']
# dropping the unnecessary features


In [1398]:
# FieldDrop1 = ['Purchase']

In [1399]:
# df = df.drop(FieldDrop1, axis=1)

In [1400]:
# df

In [1401]:
len(Feature_Drop) # these are features that are absolutely certain to be removed

0

In [1402]:
len(Feature_DropSoft) # these are features that are not certain to be removed

2

In [1403]:
# Feature_DropSoft = Feature_DropSoft + Feature_Drop
# len(Feature_DropSoft)

In [1404]:
UFeatures_Drop = list(())
UFeatures_DropSoft = list(())

In [1405]:
[UFeatures_Drop.append(x) for x in Feature_Drop if x not in UFeatures_Drop]
[UFeatures_DropSoft.append(z) for z in Feature_DropSoft if z not in UFeatures_DropSoft]

[None, None]

In [1406]:
length = len(UFeatures_Drop)
lengthSoft = len(UFeatures_DropSoft)

In [1407]:
middle_index = length//2 # source code URL: https://www.kite.com/python/answers/how-to-split-a-list-in-half-in-python
middle_indexSoft = lengthSoft//2

In [1408]:
len(UFeatures_Drop)

0

In [1409]:
len(UFeatures_DropSoft)

2

In [1410]:
UFeatures_Droph = UFeatures_Drop[:middle_index]
UFeatures_DropSofth = UFeatures_DropSoft[:middle_indexSoft]

In [1411]:
len(UFeatures_Droph)

0

In [1412]:
len(UFeatures_DropSofth)

1

In [1413]:
UFeatures_Drop_United = UFeatures_Droph + UFeatures_DropSofth

In [1414]:
len(UFeatures_Drop_United)

1

In [1415]:
# df

In [1416]:
newdf = df.drop(UFeatures_Drop_United, axis=1).copy()

In [1417]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497996 entries, 0 to 497995
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Marital_Status                 497996 non-null  float64
 1   Product_Category_1             497996 non-null  float64
 2   Product_Category_2             497996 non-null  float64
 3   Purchase                       497996 non-null  float64
 4   Gender_M                       497996 non-null  float64
 5   Age_0-17                       497996 non-null  float64
 6   Age_18-25                      497996 non-null  float64
 7   Age_26-35                      497996 non-null  float64
 8   Age_36-45                      497996 non-null  float64
 9   Age_46-50                      497996 non-null  float64
 10  Age_51-55                      497996 non-null  float64
 11  Age_55+                        497996 non-null  float64
 12  Occupation_0                  

In [1418]:
colnamesP = newdf.columns.values.tolist()
DropColPurchase = ['Purchase']
Purchase = str('Purchase')

In [1419]:
for sublist in colnamesP:
  if Purchase in sublist:
    newdf = newdf.drop(DropColPurchase, axis=1).copy()
  else:
    pass

In [1420]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497996 entries, 0 to 497995
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Marital_Status                 497996 non-null  float64
 1   Product_Category_1             497996 non-null  float64
 2   Product_Category_2             497996 non-null  float64
 3   Gender_M                       497996 non-null  float64
 4   Age_0-17                       497996 non-null  float64
 5   Age_18-25                      497996 non-null  float64
 6   Age_26-35                      497996 non-null  float64
 7   Age_36-45                      497996 non-null  float64
 8   Age_46-50                      497996 non-null  float64
 9   Age_51-55                      497996 non-null  float64
 10  Age_55+                        497996 non-null  float64
 11  Occupation_0                   497996 non-null  float64
 12  Occupation_1                  

In [1421]:
# newdf2 = newdf.drop(Feature_DropSoft, axis=1).copy()

In [1422]:
# newdf2.shape()

In [1423]:
# newtr1 = newtr[['Gender_F','Gender_M','Age_0-17','Age_18-25','Age_26-35','Age_36-45','Age_46-50','Age_51-55','Age_55+','City_Category_A','City_Category_B','City_Category_C','Stay_In_Current_City_Years_0','Stay_In_Current_City_Years_1','Stay_In_Current_City_Years_2','Stay_In_Current_City_Years_3','Stay_In_Current_City_Years_4+']].copy()

In [1424]:
# models = [LinearRegression(), # метод наименьших квадратов
#	          RandomForestRegressor(n_estimators=100, max_features ='sqrt'), # случайный лес
#	          KNeighborsRegressor(n_neighbors=6), # метод ближайших соседей
#	          SVR(kernel='linear'), # метод опорных векторов с линейным ядром
#	          LogisticRegression() # логистическая регрессия
#	          ]



In [1425]:
from sklearn.model_selection import train_test_split

In [1426]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(newdf, target, test_size=0.2)


In [1427]:
print (Xtrn.shape)
print (Ytrn.shape)
print (Xtest.shape)
print (Ytest.shape)
# print X_test.shape, y_test.shape
# source code URL: https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

(398396, 40)
(398396,)
(99600, 40)
(99600,)


In [1428]:
# fit a model
lm = LinearRegression()
model = lm.fit(Xtrn, Ytrn)
predictions = lm.predict(Xtest)


In [1429]:
# Let’s see what (some of) the predictions are:
predictions[0:5]

array([11281.5  ,  9332.5  , 11526.625, 10879.375, 11128.   ])

In [1430]:
Ytest = Ytest.astype(float)
predictions = predictions.astype(float)

In [1431]:
# source code URL: https://python-school.ru/linear-regression-basis/

# Ytest = np.exp(Ytest)
# predictions = np.exp(predictions)

In [1432]:
# source code URL: https://python-school.ru/linear-regression-basis/
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mse = mean_squared_error(Ytest, predictions)
mae = mean_absolute_error(Ytest, predictions)
print('Mean Squared Error: %.3f, Mean Absolute Error: %.3f' % (mse, mae))

Mean Squared Error: 22196262.608, Mean Absolute Error: 3627.700


In [1433]:
# import seaborn as sns

In [1434]:
# sns.pairplot(newdf)


In [1435]:
# features = newdf.columns.values.tolist()

In [1436]:
# sns.pairplot(newdf, x_vars=[features], y_vars='predictions', kind='reg', height=100)